In [1]:
import pandas as pd
import numpy as np
import torch 

import warnings
warnings.filterwarnings("ignore")

In [2]:
# 录入最原始的数据
idxc = pd.read_csv("../task3/data/index_component.csv")
rf = pd.read_csv("../task3/data/risk_factor.csv")
y_pred = pd.read_csv("../task3/data/y_pred.csv")
y_true = pd.read_csv("../task3/data/y_true.csv")

In [3]:
# 按trade_date,stock_code排序
idxc = idxc.sort_values(['trade_date','stock_code'])
rf = rf.sort_values(['trade_date','stock_code'])
y_pred = y_pred.sort_values(['trade_date','stock_code'])
y_true = y_true.sort_values(['trade_date','stock_code'])

In [4]:
# rf表删除重复行
rf = rf.drop_duplicates()

In [5]:
# risk_factor_5,6,8的值全为零，不包含任何信息，所以要删除
rf.drop(['risk_factor_5','risk_factor_6','risk_factor_8'],axis=1,inplace=True)

In [10]:
# 因为idxc表含有stock_code=2754的数据，但y_true表却没有,从而没法算收益
# 又观察到idxc表中这一股票的数据只到trade_date=3422, 所以只需删除trade_date<=3422之前四张表的所有数据
idxc.drop(idxc[idxc['trade_date']<=3422].index,inplace=True)
rf.drop(rf[rf['trade_date']<=3422].index,inplace=True)
y_pred.drop(y_pred[y_pred['trade_date']<=3422].index,inplace=True)
y_true.drop(y_true[y_true['trade_date']<=3422].index,inplace=True)

In [17]:
# y_true,y_pred表只到trade_date=3762,而rf,idxc表到trade_date=3981
# 但trade_date=3762这一天冒出若干新股数据, 所以把四张表中trade_date>=3762的数据删除
idxc.drop(idxc[idxc['trade_date']>=3762].index,inplace=True)
rf.drop(rf[rf['trade_date']>=3762].index,inplace=True)
y_pred.drop(y_pred[y_pred['trade_date']>=3762].index,inplace=True)
y_true.drop(y_true[y_true['trade_date']>=3762].index,inplace=True)

In [21]:
# 对于stock_code=97,idxc表有3423<=trade_date<=3518的数据，但y_true表却只有3423<=trade_date<=3511的数据
# 而且观察到y_true表中3507<=trade_date<=3511（最后5天）的数据是一样的，那么我们可以将这个值延拓到trade_date=3518
# 至此,idxc表中所有（trade_date，stock_code）都能在y_true表中找到,从而可以计算收益
for i in range(7): # 一共要补7天的数据
    temp = y_true[y_true['stock_code']==97].iloc[-1,:]
    temp['trade_date']+=1
    y_true = y_true.append(temp)
y_true = y_true.sort_values(['trade_date','stock_code'])

In [22]:
# 检查idxc表中所有(trade_date，stock_code)是否都能在y_true表中找到
date_idx = np.sort(np.array(list(set(y_true['trade_date']))))
code_idx = np.sort(np.array(list(set(y_true['stock_code']))))
delete_date = []
for date in date_idx:
    ret_code = set(y_true[y_true['trade_date']==date]['stock_code'])
    idxc_code = set(idxc[idxc['trade_date']==date]['stock_code'])
    if len(idxc_code-ret_code)>0:
        delete_date.append(date)
print(delete_date)

[]


In [23]:
y_true['stock_code'] = y_true['stock_code'].astype(int)
y_true['trade_date'] = y_true['trade_date'].astype(int)

In [24]:
idxc.to_csv('../task3/clean_data/clean_idxc.csv',index=False)
rf.to_csv('../task3/clean_data/clean_rf.csv',index=False)
y_pred.to_csv('../task3/clean_data/clean_pred.csv',index=False)
y_true.to_csv('../task3/clean_data/clean_true.csv',index=False)